In [ ]:
# 경복궁_tripadvisor

from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from tqdm import tqdm

comment = []
score = []
date = []

url = 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d324888-Reviews-or00-Gyeongbokgung_Palace-Seoul.html'
req = requests.get(url, headers = headers)
target = BeautifulSoup(req.content.decode('utf-8', 'replace'), 'html.parser')
page = int(re.sub(',','',target.find(class_='cIUfa Ci').get_text().split('\xa0')[1][:-2]))

if page %10 != 0:
    page_n = page//10 +1
else:
    page_n = page//10

    
for i in tqdm(range(page_n)):
    url = f'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d324888-Reviews-or{i}0-Gyeongbokgung_Palace-Seoul.html'
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    req = requests.get(url, headers = headers)
    target = BeautifulSoup(req.content.decode('utf-8', 'replace'), 'html.parser')
    reviews = target.select('#tab-data-qa-reviews-0 > div > div.bPhtn > div')
    
    for review in reviews:
        if review.find(class_='dYkZQ d') or review.find(class_='dpHcY'):
            continue
            
            
        a = review.find('div',class_ ='WlYyy diXIH dDKKM')
        b = review.find('svg',class_="RWYkj d H0")
        c = review.find(class_='fEDvV')
        
        for data, li in zip([a,b,c], [comment, score, date]):
            if data:
                if data == a or data==c:
                    li.append(data.text)
                else:
                    li.append(data['aria-label'].split(' ')[-1])
                
            else:
                li.append(None)
            

        
data = pd.DataFrame({'comment':comment, 'score':score, 'date':date})
data.to_csv('trip/tripadvisor_경복궁.csv')
data

In [ ]:
# 경복궁_네이버 플레이스
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm


def delete_iframe( url):
    res_code = re.findall(r"place/(\d+)", url)
    final_url = 'https://m.place.naver.com/place/'+res_code[0]+'/review/visitor#'
    return final_url

url = 'https://map.naver.com/v5/entry/place/11571707?placePath=%2Freview%2Fvisitor%3Fentry=plt&c=14134551.0028198,4520099.7994413,15,0,0,0,dh'
# url = 'https://map.naver.com/v5/search/%EA%B2%BD%EB%B3%B5%EA%B6%81/place/11784606?c=14133850.9368061,4519794.5765119,15,0,0,0,dh&placePath=%3Fentry%253Dbmp'
new_url = delete_iframe(url)

driver = webdriver.Chrome('webdriver/chromedriver.exe')


driver.get(new_url)

while True:
        try:
            driver.implicitly_wait(10)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            driver.implicitly_wait(10)
            driver.find_element_by_link_text('더보기').click()
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break
            
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# 네이버 추가 코드
from tqdm import tqdm

df = pd.DataFrame(columns=['comment','score','date'])
for reviews in tqdm(soup.select('li._2Cv-r')):
    li = [None, None, None]
    try:
        li[0] = reviews.find(class_='WoYOw').get_text()
    except:
        pass
    li[1] = reviews.find(class_='_2tObC').get_text()
    li[2] = reviews.select('div.ZvQ8X > span')[0].get_text()
    series = pd.Series(li,index=['comment','score','date'])
    df = df.append(series, ignore_index=True)

df['score'] = df['score'].astype('float32')
df['date'] = pd.to_datetime(df['date'])
df

In [ ]:
# 네이버 업데이트 버전 (10/26 이후)
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm

def delete_iframe(url):
    res_code = re.findall(r"place/(\d+)", url)
    final_url = 'https://m.place.naver.com/place/'+res_code[0]+'/review/visitor#'
    return final_url

url = 'https://map.naver.com/v5/entry/place/11583237?c=14139229.5383789,4499752.9660836,15,0,0,0,dh'
# url = 'https://map.naver.com/v5/search/%EA%B2%BD%EB%B3%B5%EA%B6%81/place/11784606?c=14133850.9368061,4519794.5765119,15,0,0,0,dh&placePath=%3Fentry%253Dbmp'
new_url = delete_iframe(url)

driver = webdriver.Chrome('D:/tool/chromedriver.exe')
driver.get(new_url)

while True:
        try:
            time.sleep(1)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
            driver.find_element_by_class_name('_3iTUo').click()
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break
            
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# 추가 코드
from tqdm import tqdm
# html = driver.page_source
# soup = BeautifulSoup(html, 'lxml')
df = pd.DataFrame(columns=['comment','score','date'])
for reviews in tqdm(soup.select('ul._1jVSG > li')):
    li = [None, None, None]
    # 리뷰 없는 경우 분리해서 크롤링
    try:
        li[0] = reviews.find(class_='WoYOw').get_text()
    except:
        pass
    # 별점 없는 경우 분리해서 크롤링
    try:    
        li[1] = reviews.find('em').get_text()
    except:
        pass
    li[2] = reviews.find('time').get_text()
    series = pd.Series(li,index=['comment','score','date'])
    df = df.append(series, ignore_index=True)

# df['score'] = df['score'].astype('float32')
# df['date'] = pd.to_datetime(df['date'])
df

In [ ]:
# 경복궁_구글맵_모바일

options = webdriver.ChromeOptions()
# options.add_argument('user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 15_0_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36')
driver = webdriver.Chrome('webdriver/chromedriver.exe',options=options)


url = 'https://www.google.com/maps/place/%EA%B2%BD%EB%B3%B5%EA%B6%81/@37.579617,126.8369653,11z/data=!4m11!1m2!2m1!1z6rK967O16raB!3m7!1s0x357ca2c74aeddea1:0x8b3046532cc715f6!8m2!3d37.579617!4d126.977041!9m1!1b1!15sCgnqsr3rs7XqtoGSAQZwYWxhY2U?hl=ko'
driver.get(url)
prev_height = driver.execute_script('return document.body.scrollHeight')
time.sleep(7)
while 1:
    scrollable_div = driver.find_element_by_css_selector('#pane > div > div.widget-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc')
    scrollable_div.send_keys(Keys.END)
    # 페이지 로딩 대기
    
    curr_height = driver.execute_script('return arguments[0].scrollHeight',scrollable_div)
    while 1:
        if prev_height==curr_height:
            time.sleep(1)
            curr_height = driver.execute_script('return arguments[0].scrollHeight',scrollable_div)
        else:
            prev_height = curr_height
            break
    while 1:
        try:
            driver.find_element_by_class_name('ODSEW-KoToPc-ShBeI.gXqMYb-hSRGPd').click()
        except:
            break
    
    try:
        driver.find_element_by_class_name('wo1ice-loading.noprint')
    except:
        break
    


        
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# 경복궁_구글맵_웹
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import time

driver = webdriver.Chrome('C:/tool/chromedriver.exe')

# https://www.google.com/travel/ 에 관광지명 검색 후
# 리뷰 클릭하여 나오는 새 창 주소로 아래 URL 바꿔야 함
# 파일명 바꿔야 함
url = 'https://www.google.com/search?hl=ko-KR&gl=kr&q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EA%B0%95%EB%82%A8%EA%B5%AC+%EC%82%BC%EC%84%B11%EB%8F%99+%EC%98%81%EB%8F%99%EB%8C%80%EB%A1%9C+513+%EC%BD%94%EC%97%91%EC%8A%A4+%EC%95%84%EC%BF%A0%EC%95%84%EB%A6%AC%EC%9B%80&ludocid=8897176098279649643&lsig=AB86z5WESsu2tVXa5ySZB0HTSYta&ved=0CBAQ9aIFahcKEwjYgqvLs-fzAhUAAAAAHQAAAAAQBQ#lrd=0x357ca46a4cb562ef:0x7b791e85aa0df16b,1'
filename = 'google_코엑스아쿠아리움.csv'   


driver.get(url)
driver.implicitly_wait(10)
scrollable_div = driver.find_element_by_css_selector('div.review-dialog-list')

for i in range(99):
    prev_height = driver.execute_script('return arguments[0].scrollHeight', scrollable_div)
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    
    while True:
        if driver.execute_script('return arguments[0].scrollHeight', scrollable_div) == prev_height:
            time.sleep(1)
        else:
            break

from bs4 import BeautifulSoup
bs = BeautifulSoup(driver.page_source, 'html.parser')

# 별점
ratings = []
for rating in bs.select('div.PuaHbe > g-review-stars > span'):
    # 숫자만 구하기
    ratings.append(rating['aria-label'][-4:-1])

print('리뷰 개수 :', len(ratings))

contents = []
for content in bs.select('div.Jtu6Td > span'):
    contents.append(content.text)

dates = []
for date in bs.select('div.PuaHbe > span.dehysf.lTi8oc'):
    dates.append(date.text)

import pandas as pd

df = pd.DataFrame({'comment' : contents, 'score' : ratings, 'date' : dates})
df.score = df.score.astype('float')


df.to_csv(filename, index = False)

In [ ]:
# 경복궁 구글맵 웹 (2)
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
from tqdm import tqdm 
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait





options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36')
options.add_argument('--incognito')
driver = webdriver.Chrome('chromedriver.exe',options=options)

url = 'https://www.google.com/search?hl=ko-KR&gl=kr&q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EC%84%B8%EC%A2%85%EB%A1%9C+%EC%82%AC%EC%A7%81%EB%A1%9C+161+%EA%B2%BD%EB%B3%B5%EA%B6%81&ludocid=10029593693201569270&lsig=AB86z5XxAP4T01gIRsz7SJ6Ajl4D&ved=0CA0Q9aIFahcKEwjQoPSotOfzAhUAAAAAHQAAAAAQBA#lrd=0x357ca2c74aeddea1:0x8b3046532cc715f6,1'

driver.get(url)
time.sleep(7)
prev_height = driver.execute_script('return document.body.scrollHeight')
scrollable_div = driver.find_element_by_class_name('review-dialog-list')

while 1:
    driver.delete_all_cookies()


    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    try:
        driver.find_element_by_class_name('jfk-activityIndicator')
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CLASS_NAME,'jfk-activityIndicator')))
        time.sleep(2)
    except :
        break
        


        
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


df = pd.DataFrame(columns=['comment','score','date'])

for review in tqdm(soup.find_all('div',class_='WMbnJf vY6njf gws-localreviews__google-review')):
    li = [None, None, None]
    li[1] = review.find(class_="Fam1ne EBe2gf")['aria-label'].split()[-1][:-1]
    li[2] = review.find(class_="dehysf lTi8oc").get_text()

    if review.find(class_='review-full-text'):
        li[0] = review.find(class_='review-full-text').get_text()
    else:
        li[0] = review.find(class_='Jtu6Td').get_text()
    
    series = pd.Series(li,index=['comment','score','date'])
    df = df.append( series,ignore_index=True)
df

df.to_csv('dataframe/google_경복궁.csv',index=False)